你有很多有用的方法，想用它来拓展其他类的功能，但是这些类并没有任何的继承关系，因此不能将这些方法放入一个基类，然后被其他类继承

通常想要自定义类的时候会碰上这种问题，可能是某个库提供了一些基础类，你可以利用他们来构造自己的类。   
假设你想拓展映射对象，想给他们添加日志、唯一性设置、类型检查等的功能，这种时候可以使用**混入类**来实现：

In [1]:
class LoggedMappingMixin:
    """ 添加log
    """
    # 混入类通常没有实力变量，因为直接实例化混入类没有任何意义
    __slots__ = ()
    
    def __getitem__(self, key):
        print('Getting:',str(key))
        return super().__getitem__(key)
    
    def __setitem__(self, key, value):
        print('Setting {} to {}'.format(str(value), str(key)))
        return super().__setitem__(key, value)
    
    def __delitem__(self, key):
        print('Delete :{}'.format(str(key)))
        return super().__delitem__(key)

    
class SetOnceMappingMixin:
    """每个键只能设置一次
    """
    __slots__ = ()
    
    def __setitem__(self, name, value):
        if key in self:
            raise KeyError(str(key) + 'already set')
        return super().__setitem__(key, value)


class StringKeyMappingMixin:
    """限制key只能为str类型
    """
    __slots__ = ()
    
    def __setitem__(self, name, value):
        if not isinstance(name, str):
            raise TypeError("Except a string")
        return super().__setitem__(name, value)

混入类单独使用没有任何意义，事实上如果你去实例化任何一个类，除了产生异常外没有任何作用。
**他们是通过多继承和其他映射对象混入使用的**

In [2]:
class LoggedDict(LoggedMappingMixin, dict):
    pass

d = LoggedDict()
d['x'] = 23
print (d['x'])

Setting 23 to x
Getting: x
23


混入类在标准库中很多地方都有出现，通常都是**用来拓展某些类的功能**   
他们也是多继承的一个主要用途。  
例如：   
    当编写网络代码是，经常会使用到socketserver模块中的 ThreadingMixin 来给其他网络相关类增加多线程支持。

In [3]:
from xmlrpc.server import SimpleXMLRPCDispatcher
from socketserver import ThreadingMixIn

class ThreadedXMLRPCServer(ThreadingMixIn, SimpleXMLRPCDispatcher):
    pass

对于混入类，需要注意的是：   
1、混入类不能直接被实例化使用   
2、混入类没有自己的状态信息，也就是说没有定义`__init__`方法，并且没有实例属性。这也是我们为什么在上面明确定义了`__slots__ = ()`

另外一种实现混入类的方法是**使用类装饰器**

In [5]:
def LoggedMapping(cls):
    cls_getitem = cls.__getitem__
    cls_setitem = cls.__setitem__
    cls_delitem = cls.__delitem__
    
    def __getitem__(self, key):
        print('Getting ' + str(key))
        return cls_getitem(self, key)

    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return cls_setitem(self, key, value)

    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return cls_delitem(self, key)

    cls.__getitem__ = __getitem__
    cls.__setitem__ = __setitem__
    cls.__delitem__ = __delitem__
    return cls

@LoggedMapping
class LoggedDict(dict):
    pass
    